模型的加载和保存
完整的模型训练套路
GPU训练
完整的模型验证套路

# 准备数据集

In [2]:
import torchvision

In [3]:
# 准备数据集
train_data = torchvision.datasets.CIFAR10(root='./data',train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10(root='./data',train=False,transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified
Files already downloaded and verified


# 创建数据集

In [4]:
train_data_size = len(train_data)
test_data_size = len(test_data)
train_data_size,test_data_size

(50000, 10000)

# 创建Dataloader

In [5]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

# 搭建网络

In [6]:
import torch.nn as nn
class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
        )

    def forward(self,x):
        x = self.model(x)
        return x

# 验证一下网络创建的是否正确

In [7]:
import torch
input = torch.ones((64,3,32,32)) # batch_size | channel_size | 行 | 列        其实batch_size=64就代表着64个图片
test = myNet()
output = test(input)
output

tensor([[-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  0.1151,  0.1097,  0.0169,  0.0112, -0.1060,  0.1080, -0.0322,
          0.2316,  0.1897],
        [-0.0130,  

网络创建测试成功
创建网络模型
定义损失函数

# GPU训练
去这个网站里面https://pytorch.org/
找到安装命令：


模型上cuda
损失函数上cuda
数据上cuda

False

In [8]:
from torch.utils.tensorboard import SummaryWriter

# 创建网络模型
model = myNet()
model = model.cuda() # 使用GPU
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
#优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# 设置训练参数
total_train_step = 0 # 记录训练次数
total_test_step = 0 # 记录测试次数
epoch = 10 # 训练轮数

# 添加tensorboard ----- 类似于日志
writer = SummaryWriter("../logs_train")

# 开始训练
model.train()
for i in range(epoch):
    print('-'*7,"第{}轮训练开始".format(i+1),'-'*7)

    # 训练步骤开始
    for data in train_dataloader:
        imgs,labels = data
        imgs = imgs.cuda()
        labels = labels.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs,labels)
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if(total_train_step%100==0):
            print('训练次数：{},loss：{}'.format(total_train_step,loss.item())) # loss.item()  ： loss原本是一个tensor 但是 使用了 item()之后就将tensor转化成了真实的东西。
            writer.add_scalar("train_loss",loss.item(),total_train_step)


    # 使用测试集
    model.eval()
    total_test_loss = 0 # 计算一下整体测试集上面的loss
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,labels = data
            imgs = imgs.cuda()
            labels = labels.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs,labels)
            total_test_loss += loss
            accuracy = (outputs.argmax(1)==labels).sum()
            total_accuracy += accuracy
    print(f'整体数据集上的loss:{total_test_loss}')
    print(f"整体测试集上的正确率：{total_accuracy/test_data_size}")
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,total_test_step)
    total_test_step+=1

    torch.save(model,f'model_{i}.pth')
    print("模型已保存")

writer.close()

AssertionError: Torch not compiled with CUDA enabled

# Softmax后   accuracy的使用

In [ ]:
outputs = torch.tensor(
[[0.1,0.2],
[0.3,0.4]]
)
pred = outputs.argmax(1)
targets = torch.tensor([0,1])
print((pred == targets).sum()) # 计算对应位置相等的个数